In [108]:
from metrcis import combined_score
import pandas as pd
import json

In [109]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Target

In [110]:
reviews_test_true = pd.read_csv('reviews_test_true.csv')
reviews_test_true.head()

,mark,date,profile_url,detail_id
0,4,2024-09-06 00:00:00,/Profile/253marinap,1418748
1,4,2024-08-26 00:00:00,/Profile/253marinap,6686078
2,5,2024-08-25 00:00:00,/Profile/253marinap,15083876
3,5,2024-08-23 00:00:00,/Profile/253marinap,9594542
4,3,2024-08-19 00:00:00,/Profile/253marinap,10298181


In [111]:
def group_details_by_profile(df):
    profile_to_details = df.groupby('profile_url')['detail_id'].apply(list).to_dict()
    return profile_to_details

reviews_test_true_dict = group_details_by_profile(reviews_test_true)

In [112]:
reviews_test_true_dict

{'/Profile/0wrainb0w': [10205971, 10234858, 10229444, 10188034, 10147203],
 '/Profile/121nadyam': [2342045, 8514577, 524833, 9753823, 11893476],
 '/Profile/2000RIVERSIDE2000': [10447143,
  10448151,
  10439876,
  9983942,
  9975382],
 '/Profile/253marinap': [1418748, 6686078, 15083876, 9594542, 10298181],
 '/Profile/354andrey': [6494352, 547839, 5776923, 6493554, 6785214],
 '/Profile/379garryv': [12012566, 6378905, 16726869, 6750854, 11549325],
 '/Profile/398tatyanap': [300383, 524833, 524791, 574653, 15083873],
 '/Profile/440nikitab': [6490293, 2561543, 2341889, 541859, 6686078],
 '/Profile/454luise': [530940, 7363113, 574164, 15347696, 6650405],
 '/Profile/5021960': [3762041, 2419130, 10431014, 10379451, 13558469],
 '/Profile/5045_': [7273038, 8680790, 8388913, 9455470, 7912817],
 '/Profile/525alexanderp': [16669086, 10005289, 300396, 6110458, 739994],
 '/Profile/580ninay': [15037680, 13002491, 6111665, 2342024, 2569510],
 '/Profile/689in_ss': [2556183, 7733344, 2557844, 300376, 2560

In [113]:
def calculate_average(data, cnt_empty):
    if not data: 
        return 0
    total_sum = sum(data.values()) 
    print(total_sum)
    count = len(data) 
    print(count)
    return total_sum / (count - cnt_empty)

### GNN

In [114]:
file_path = "recommendations_gnn.json"

with open(file_path, 'r', encoding='utf-8') as file:
    data_gnn = json.load(file)

In [116]:
gnn_score = {}
cnt_empty = 0

for user, ids in data_gnn.items():
    if not ids:
        cnt_empty += 1
        continue
    visited_features = items[items['detail_id'].isin(ids)] 
    true_ids = reviews_test_true_dict[user]

    score = combined_score(ids, true_ids, items, alpha=0.7, beta=0.3, visited_features=visited_features)
    print(f"Combined Score user {user}: {score}")
    gnn_score[user] = score

Combined Score user /Profile/253marinap: 0.6299219173213028
Combined Score user /Profile/Insta_natytravel: 0.635162304519958
Combined Score user /Profile/Talant2007: 0.7836706412543687
Combined Score user /Profile/smolkin: 0.807961119007224
Combined Score user /Profile/W7946AG_: 0.7478057153007153
Combined Score user /Profile/sergeyk147: 0.819864581841252
Combined Score user /Profile/440nikitab: 0.7522278693370634
Combined Score user /Profile/Nastroyshchik: 0.6344804511478476
Combined Score user /Profile/Svetlanka_Me: 0.622984719195255
Combined Score user /Profile/darovs: 0.5887746494896959
Combined Score user /Profile/oleg19660320: 0.26869039397036254
Combined Score user /Profile/AlexeevIgor: 0.6449998219345782
Combined Score user /Profile/_F8969HH: 0.6408303589446896
Combined Score user /Profile/2000RIVERSIDE2000: 0.5900229860553806
Combined Score user /Profile/0wrainb0w: 0.5925510373567325
Combined Score user /Profile/itacherne: 0.7208186566950043
Combined Score user /Profile/Q1783K

In [117]:
gnn_score_ans = float(calculate_average(gnn_score, cnt_empty))
gnn_score_ans

130.7615962836916
199


0.6604121024428868

In [118]:
# функция для подсчета скора
def calculate_average_for_file(file_path, items, reviews_test_true_dict, alpha=0.7, beta=0.3):
    def calculate_average(data, cnt_empty):
        total = sum(data.values())
        return total / (len(data) - cnt_empty) if len(data) + cnt_empty > 0 else 0
    
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    scores = {}
    cnt_empty = 0

    for user, ids in data.items():
        if not ids:
            cnt_empty += 1
            continue
        
        visited_features = items[items['detail_id'].isin(ids)]
        true_ids = reviews_test_true_dict[user]

        score = combined_score(ids, true_ids, items, alpha, beta, visited_features=visited_features)
        scores[user] = score

    score = float(calculate_average(scores, cnt_empty))
    return score

In [119]:
gnn_score_result = calculate_average_for_file("recommendations_gnn.json", items, reviews_test_true_dict)
gnn_score_result

0.6604121024428868

### Recommend route

In [120]:
rec_route_score_result = calculate_average_for_file("recommendations_rec_route.json", items, reviews_test_true_dict)
rec_route_score_result

0.45518962830508114

### PageRank

In [121]:
pagerank_score_result = calculate_average_for_file("recommendations_pagerank.json", items, reviews_test_true_dict)
pagerank_score_result

0.48614285986819117

### Use tags

In [122]:
use_tags_score_result = calculate_average_for_file("use_tags.json", items, reviews_test_true_dict)
use_tags_score_result

0.5016859527694396

### Similar users

In [123]:
similar_users_score_result = calculate_average_for_file("similar_users.json", items, reviews_test_true_dict)
similar_users_score_result

0.6424820024904798

### User similarity graph

In [124]:
graph_recommendation_score_result = calculate_average_for_file("graph_similar_users.json", items, reviews_test_true_dict)
graph_recommendation_score_result

0.6279265370349887

### Graph recommendations

In [125]:
graph_recommendations__score_result = calculate_average_for_file("graph_recommendations_.json", items, reviews_test_true_dict)
graph_recommendations__score_result

0.31230176905506263

### Random walk

In [126]:
random_walk_recommendations_score_result = calculate_average_for_file("random_walk_recommendations.json", items, reviews_test_true_dict)
random_walk_recommendations_score_result

0.45702689076230774

### Lasso predict rating

In [128]:
lasso_recommendations_score_result = calculate_average_for_file("lasso_recommendations.json", items, reviews_test_true_dict)
lasso_recommendations_score_result 

0.6237266764944386

### LightGCN

In [129]:
lightgcn_score_result = calculate_average_for_file("recommendations_lightgcn_1.json", items, reviews_test_true_dict)
lightgcn_score_result 

0.642603021681348

### Lasso

In [131]:
lasso_20_score_result = calculate_average_for_file("20_lasso.json", items, reviews_test_true_dict)
lasso_20_score_result 

0.640264098591127